In [ ]:
# import required packages
from __future__ import print_function, division
#import cv2
import dlib
import time
from skimage import io
import os
import shutil
import torch
import pandas as pd
import torchvision
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.stats.api as sms
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from os import listdir
from os.path import isfile, join
from collections import namedtuple
from skimage.transform import rescale
from skimage.transform import resize
import sys
import glob
import PIL
import os
import shutil
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn.functional as F
import math
import PIL
import imgaug as ia
from imgaug import augmenters as iaa
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
from pathlib import Path
from collections import Counter
import imgaug as ia
from imgaug import augmenters as iaa
import cv2
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode
print(os.getcwd())
#np.random.seed(198467)
torch.cuda.empty_cache()

In [2]:
frame = pd.read_csv('ExpW_OpenFace_result_with_expression_gender_race_age.csv')
AU_intensity_OpenFace_common2 = ['AU01_r', 'AU02_r', 'AU04_r', 'AU05_r', 'AU06_r', 'AU09_r', 'AU10_r', 
                'AU12_r', 'AU15_r', 'AU17_r', 'AU20_r', 'AU25_r', 'AU26_r']
optimal_threshold = np.array([4.4, 4.8, 2.4, 3.7, 1.8, 2.8, 0.2, 1.3, 1.3, 5. , 5. , 0.7, 5. ])
for i in range(len(AU_intensity_OpenFace_common2)):
    frame[AU_intensity_OpenFace_common2[i] + "_c"] = frame[AU_intensity_OpenFace_common2[i]].apply(lambda x: 0 if x <= optimal_threshold[i] else 1)

frame = frame.dropna()
frame = frame[['ImageName', 'Expression', 'gender_preds', 'AU06_r_c', 'AU12_r_c']]
frame["Expression"] = frame["Expression"].apply(lambda x: 1 if x == 3 else 0)
frame = frame.dropna()
frame['AU'] = frame.AU06_r_c.astype(int).astype(str) + frame.AU12_r_c.astype(int).astype(str)
def convert_AU_to_4types(s):
    if s == '00': return 0
    elif s == '10': return 1
    elif s == '01': return 2
    elif s == '11': return 3
frame.AU = frame.AU.apply(convert_AU_to_4types)
frame.head()

,ImageName,Expression,gender_preds,AU06_r_c,AU12_r_c,AU
0,angry_actor_104_0.jpg,0,Male,1,0,1
1,angry_actor_109_0.jpg,0,Male,0,0,0
2,angry_actor_120_1.jpg,0,Male,0,0,0
3,angry_actor_13_0.jpg,1,Female,0,0,0
4,angry_actor_132_0.jpg,0,Female,0,0,0


In [3]:
frame.shape

(85355, 6)

In [4]:
frame.Expression.value_counts()

0    57103
1    28252
Name: Expression, dtype: int64

In [5]:
frame.gender_preds.value_counts()

Male      58929
Female    26426
Name: gender_preds, dtype: int64

In [6]:
frame.AU.value_counts()

0    61183
2    13396
3     8145
1     2631
Name: AU, dtype: int64

In [7]:
frame['ImageName'] = frame['ImageName'].apply(lambda x: '/data/ExpW_cropped_by_their_coordinates/' + x) # change ImageName to full path
frame_copy = frame.copy()

In [8]:
result_rows_list = []

In [9]:
for k in range(1,6):

    # Split into train/validation/test sets
    frame = frame_copy.sample(n = 20000, random_state = k).reset_index(drop=True) # shuffle data frame
    n_images = len(frame)
    n_train = int(0.8 * n_images)
    n_val = int((n_images - n_train) / 2)
    n_test = n_images - n_train - n_val

    train_frame = frame[0 : n_train].reset_index(drop=True)
    val_frame = frame[n_train : n_train + n_val].reset_index(drop=True)
    test_frame = frame[n_train + n_val : ].reset_index(drop=True)

    print("{} train faces, {} validation faces, {} test faces".format(len(train_frame), len(val_frame), len(test_frame)))



    # Data loaders and transforms for training

    # Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
    # e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second
    # image.
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)

    # Define our sequence of augmentation steps that will be applied to every image.
    class ImgAugTransform:

        def __init__(self):
            self.aug = iaa.Sequential(
            [
                #
                # Apply the following augmenters to most images.
                #
                iaa.Fliplr(0.5), # horizontally flip 50% of all images
                #iaa.Flipud(0.2), # vertically flip 20% of all images

                # crop some of the images by 0-10% of their height/width
                sometimes(iaa.Crop(percent=(0, 0.05))),

                # Apply affine transformations to some of the images
                # - scale to 80-120% of image height/width (each axis independently)
                # - translate by -20 to +20 relative to height/width (per axis)
                # - rotate by -45 to +45 degrees
                # - shear by -16 to +16 degrees
                # - order: use nearest neighbour or bilinear interpolation (fast)
                # - mode: use any available mode to fill newly created pixels
                #         see API or scikit-image for which modes are available
                # - cval: if the mode is constant, then use a random brightness
                #         for the newly created pixels (e.g. sometimes black,
                #         sometimes white)
                iaa.Affine(
                    scale={"x": (1, 1.1), "y": (1, 1.1)}, # scale images to 80-120% of their size, individually per axis
                    translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, # translate by -10 to +10 percent (per axis)
                    rotate=(-15, 15), # rotate by -15 to +15 degrees
                    shear=(-8, 8), # shear by -8 to +8 degrees
                    order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
                    #cval=(0, 255), # if mode is constant, use a cval between 0 and 255
                    mode=['edge'] # use any of scikit-image's warping modes (see 2nd image from the top for examples)
                ),

                #
                # Execute 0 to 5 of the following (less important) augmenters per
                # image. Don't execute all of them, as that would often be way too
                # strong.
                #
                iaa.SomeOf((0, 5),
                    [
                        # Convert some images into their superpixel representation,
                        # sample between 20 and 200 superpixels per image, but do
                        # not replace all superpixels with their average, only
                        # some of them (p_replace).
                        sometimes(
                            iaa.Superpixels(
                                p_replace=(0, 0.1),
                                n_segments=(50, 200)
                            )
                        ),

                        # Blur each image with varying strength using
                        # gaussian blur (sigma between 0 and 3.0),
                        # average/uniform blur (kernel size between 2x2 and 7x7)
                        # median blur (kernel size between 3x3 and 11x11).
                        iaa.OneOf([
                            iaa.GaussianBlur((0, 3.0)),
                            iaa.AverageBlur(k=(2, 7)),
                            iaa.MedianBlur(k=(3, 11)),
                        ]),

                        # Sharpen each image, overlay the result with the original
                        # image using an alpha between 0 (no sharpening) and 1
                        # (full sharpening effect).
                        iaa.Sharpen(alpha=(0, 0.3), lightness=(0.75, 1.5)),

                        # Same as sharpen, but for an embossing effect.
                        iaa.Emboss(alpha=(0, 0.3), strength=(0, 2)),

                        # Search in some images either for all edges or for
                        # directed edges. These edges are then marked in a black
                        # and white image and overlayed with the original image
                        # using an alpha of 0 to 0.7.
                        sometimes(iaa.OneOf([
                            iaa.EdgeDetect(alpha=(0, 0.3)),
                            iaa.DirectedEdgeDetect(
                                alpha=(0, 0.3), direction=(0.0, 1.0)
                            ),
                        ])),

                        # Add gaussian noise to some images.
                        # In 50% of these cases, the noise is randomly sampled per
                        # channel and pixel.
                        # In the other 50% of all cases it is sampled once per
                        # pixel (i.e. brightness change).
                        iaa.AdditiveGaussianNoise(
                            loc=0, scale=(0.0, 0.05*255), per_channel=0.5
                        ),

                        # Either drop randomly 1 to 10% of all pixels (i.e. set
                        # them to black) or drop them on an image with 2-5% percent
                        # of the original size, leading to large dropped
                        # rectangles.
                        iaa.OneOf([
                            iaa.Dropout((0.01, 0.02), per_channel=0.5),
                            #iaa.CoarseDropout(
                            #    (0.03, 0.15), size_percent=(0.02, 0.05),
                            #    per_channel=0.2
                            #),
                        ]),

                        # Invert each image's chanell with 5% probability.
                        # This sets each pixel value v to 255-v.
                        #iaa.Invert(0.05, per_channel=True), # invert color channels

                        # Add a value of -10 to 10 to each pixel.
                        iaa.Add((-15, 15), per_channel=0.5),

                        # Change brightness of images (50-150% of original value).
                        iaa.Multiply((0.75, 1.25), per_channel=0.5),

                        # Improve or worsen the contrast of images.
                        iaa.ContrastNormalization((0.75, 1.75), per_channel=0.5),

                        # Convert each image to grayscale and then overlay the
                        # result with the original with random alpha. I.e. remove
                        # colors with varying strengths.
                        iaa.Grayscale(alpha=(0.0, 1.0)),

                        # In some images move pixels locally around (with random
                        # strengths).
                        #sometimes(
                        #    iaa.ElasticTransformation(alpha=(0.1, 0.2), sigma=0.25)
                        #),

                        # In some images distort local areas with varying strength.
                        sometimes(iaa.PiecewiseAffine(scale=(0.005, 0.01)))
                    ],
                    # do all of the above augmentations in random order
                    random_order=True
                )
            ],
            # do all of the above augmentations in random order
            random_order=True
        )

        def __call__(self, img):

            img = np.array(img)
            return self.aug.augment_image(img)

    class ImageDataset(Dataset):

        def __init__(self, data_frame, transform=None):

            self.data_frame = data_frame
            self.transform = transform

        def __len__(self):

            return len(self.data_frame)

        def __getitem__(self, idx):
            #idx is index from dataset
            #This is a mapping from your data_frame to the output of the mode
            img_name = self.data_frame.loc[idx, 'ImageName']

            expression = self.data_frame.loc[idx, 'Expression']
            AU = self.data_frame.loc[idx, 'AU']

            # read image as ndarray, H*W*C
            image = dlib.load_rgb_image(img_name)       
            image = cv2.resize(image, (224,224)) # resize the image to 224x224 for the ResNet Model

            if self.transform:
                image = self.transform(image)

            # transform label to torch tensor
            # This sets the order of the label
            return (image, torch.from_numpy(np.asarray(expression, dtype=np.float32)), 
                    torch.from_numpy(np.asarray(AU, dtype=np.float32)))


    transform_train_data = transforms.Compose([
        ImgAugTransform(),
        lambda x: PIL.Image.fromarray(x),
        transforms.ToTensor(), 
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    transformed_train_dataset = ImageDataset(data_frame=train_frame,
                                               transform=transform_train_data
                                               )

    train_dataloader = DataLoader(transformed_train_dataset, batch_size=32,
                            shuffle=True, num_workers=8)


    transform_val_data = transforms.Compose(([transforms.ToPILImage(), 
                                          transforms.ToTensor(), 
                                          transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                              ]))

    transformed_val_dataset = ImageDataset(data_frame=val_frame,
                                               transform=transform_val_data
                                               )

    val_dataloader = DataLoader(transformed_val_dataset, batch_size=32,
                            shuffle=True, num_workers=8)

    transform_test_data = transforms.Compose(([transforms.ToPILImage(), 
                                          transforms.ToTensor(), 
                                          transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                              ]))

    transformed_test_dataset = ImageDataset(data_frame=test_frame,
                                               transform=transform_test_data
                                               )

    test_dataloader = DataLoader(transformed_test_dataset, batch_size=32,
                            shuffle=False, num_workers=8)

    # Training

    torch.cuda.is_available()

    dataloaders = {'train': train_dataloader, 'test': test_dataloader}
    dataset_sizes = {'train': len(transformed_train_dataset), 'test': len(transformed_test_dataset)}
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    #Following cell:
    #
    #* model = pretrained resnet imagenet model
    #* criteron - loss function (cross entropy loss)
    #* optimizer - optimization algorithm (Adam)
    #* epochs - number of training epochs

    from pytorch_metric_learning import miners, losses
    class Identity(nn.Module):
        def __init__(self):
            super(Identity, self).__init__()

        def forward(self, x):
            return x

    class ResNet50_base(nn.Module):   
        """ResNet50 but without the final fc layer"""

        def __init__(self, hidden_size=2048, dropout=0.5):
            super().__init__()
            self.resnet = torchvision.models.resnet50(pretrained=True)                
            #self.resnet.fc = nn.Linear(2048, hidden_size)
            self.resnet.fc = Identity()
            #self.relu = nn.ReLU()
            #self.dropout = nn.Dropout(dropout)        

        def require_all_grads(self):
            for param in self.parameters():
                param.requires_grad = True

        def forward(self, x):
            features = self.resnet(x)
            #features = self.dropout(self.relu(features))

            return features

    class Similarity_AU_model():
        def __init__(self):


            self.training_ratio = 3
            self.alpha = 100
            self.epoch = 0
            self.best_dev_mAP = 0.
            self.train_loader = train_dataloader    
            self.dev_loader = val_dataloader        
            self.test_loader = test_dataloader



            self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


            full_model = torchvision.models.resnet50(pretrained=True)
            #self.base_network = torch.nn.Sequential(*(list(full_model.children())[:-1])).to(self.device) # Without last layer
            self.base_network = ResNet50_base().to(self.device)
            self.fc_network = nn.Linear(2048, 2).to(self.device)

            self.criterion = nn.CrossEntropyLoss()



            self.base_optimizer = torch.optim.Adam( 
                                      params=filter(lambda p: p.requires_grad, self.base_network.parameters()), 
                                      lr=1e-4)
            self.fc_optimizer = torch.optim.Adam( 
                                      params=filter(lambda p: p.requires_grad, self.fc_network.parameters()), 
                                      lr=1e-4)

        def _criterion(self, output, expression):
            #return F.binary_cross_entropy_with_logits(torch.squeeze(output), expression.float())
            return self.criterion(torch.squeeze(output), expression.long())


        def state_dict(self):
            state_dict = {
                'base_network': self.base_network.state_dict(),
                'fc_network': self.fc_network.state_dict(),
                'base_optimizer': self.base_optimizer.state_dict(),
                'fc_optimizer': self.fc_optimizer.state_dict(),
                'epoch': self.epoch
            }
            return state_dict

        def load_state_dict(self, state_dict):
            self.base_network.load_state_dict(state_dict['base_network'])
            self.fc_network.load_state_dict(state_dict['fc_network'])

        def inference(self, output):
            predict_prob = torch.sigmoid(output)
            return predict_prob.cpu().numpy()



        def _train(self, loader):
            """Train the model for one epoch"""

            self.base_network.train()
            self.fc_network.train()

            train_class_loss = 0
            total = 0
            class_correct = 0
            for i, (images, expression, AU) in enumerate(loader):
                images, expression, AU = images.to(self.device), expression.to(self.device), AU.to(self.device)

                self.base_optimizer.zero_grad()
                self.fc_optimizer.zero_grad()

                features = self.base_network(images)
                class_outputs = torch.squeeze(self.fc_network(features))

                class_loss = self._criterion(class_outputs, expression)

                miner = miners.MultiSimilarityMiner(epsilon=0.1)
                loss_func = losses.TripletMarginLoss(margin=1)
                hard_pairs = miner(features, AU)
                triplet_loss = loss_func(features, AU, hard_pairs)



                total += expression.size(0)

                #class_predicted = torch.where(torch.sigmoid(class_outputs) >= 0.5, torch.ones_like(class_outputs), torch.zeros_like(class_outputs)).long()
                _, class_predicted = class_outputs.max(1)
                class_correct += class_predicted.eq(expression.long()).sum().item()




                #if i == 0:
                #    print()
                #    print("class outputs:")
                #    print(class_outputs)
                #    print("class predicted")
                #    print(class_predicted)
                #    print("class loss")
                #    print(class_loss)
                #    print()

                #print(class_loss)
                #print(self.alpha*triplet_loss)

                loss = class_loss + self.alpha*triplet_loss
                loss.backward()
                self.fc_optimizer.step()
                self.base_optimizer.step()

                train_class_loss += loss.item()

            print('Training epoch {}: [{}|{}], class loss:{}, class accuracy: {}'
                          .format(self.epoch, i+1, len(loader), 
                                  loss.item(), 100.*class_correct/total))


            self.epoch += 1

        def _test(self, loader):
            """Compute model output on test set"""

            self.base_network.eval()
            self.fc_network.eval()

            test_class_loss = 0
            total = 0
            class_correct = 0
            feature_list = []
            class_output_list = []

            with torch.no_grad():
                for i, (images, expression, AU) in enumerate(loader):
                    images, expression, AU = images.to(self.device), expression.to(self.device), AU.to(self.device)
                    features = self.base_network(images)
                    class_outputs = torch.squeeze(self.fc_network(features))

                    class_loss = self._criterion(class_outputs, expression)

                    miner = miners.MultiSimilarityMiner(epsilon=0.1)
                    loss_func = losses.TripletMarginLoss(margin=1)
                    hard_pairs = miner(features, AU)
                    triplet_loss = loss_func(features, AU, hard_pairs)


                    #print(class_loss)
                    #print(triplet_loss)
                    #loss = class_loss + self.alpha*(loss_0 + loss_1 + loss_2 + loss_3)/(n0 + n1 + n2 + n3)
                    loss = class_loss + self.alpha*triplet_loss
                    test_class_loss += loss.item()


                    total += expression.size(0)

                    #class_predicted = torch.where(torch.sigmoid(class_outputs) >= 0.5, torch.ones_like(class_outputs), torch.zeros_like(class_outputs)).long()
                    _, class_predicted = class_outputs.max(1)
                    class_correct += class_predicted.eq(expression.long()).sum().item()

                    class_output_list.append(class_outputs)
                    feature_list.append(features)

                #print('Testing epoch, class loss:{}, class accuracy: {}, domain loss: {}, domain accuracy: {}'
                #          .format(test_class_loss, 100.*class_correct/total, test_domain_loss, 
                #                  100.*domain_correct/total))
                return test_class_loss, torch.cat(class_output_list), 100.*class_correct/total, torch.cat(feature_list)

        def public_test(self, images):
            """Compute model output on test set"""

            self.base_network.eval()
            self.fc_network.eval()


            features = self.base_network(images)
            class_outputs = torch.squeeze(self.fc_network(features))

            return F.softmax(class_outputs, dim = 0)#torch.sigmoid(class_outputs)

        def train(self):
            """Train the model for one epoch, evaluate on validation set and 
            save the best model
            """

            start_time = datetime.now()
            learning_rates = [1e-4, 1e-5, 1e-6] 
            epochs = [6, 4, 4] 

            for learning_rate, epoch in zip(learning_rates, epochs):
                print()
                print('learning rate:', learning_rate)
                print('epoch number:', epoch)
                self.base_optimizer = torch.optim.Adam( 
                                      params=filter(lambda p: p.requires_grad, self.base_network.parameters()), 
                                      lr=learning_rate)
                self.fc_optimizer = torch.optim.Adam( 
                                          params=filter(lambda p: p.requires_grad, self.fc_network.parameters()), 
                                          lr=learning_rate, weight_decay=1e-5)
                for i in range(epoch):
                    print()
                    self._train(self.train_loader)

                    dev_class_loss, dev_class_output,dev_class_accuarcy, _ = self._test(self.dev_loader)


                    print('Testing epoch, class loss:{},  class accuracy: {}'
                              .format(dev_class_loss/len(self.dev_loader), dev_class_accuarcy))


                    torch.save(self.state_dict(), os.path.join("./", 'hardtriplets_margin1_' + str(k) + '.pth'))

            duration = datetime.now() - start_time
            print('Finish training epoch {}, dev class loss: {}, time used: {}'
                  .format(self.epoch, dev_class_loss/len(self.dev_loader), duration))

        def test(self):
            # Test and save the result
            state_dict = torch.load(os.path.join("./", 'hardtriplets_margin1_' + str(k) + '.pth'))
            self.load_state_dict(state_dict)



            test_class_loss, test_class_output, test_class_accuarcy, _ = self._test(self.test_loader)
            test_predict_prob = self.inference(test_class_output)
            print(test_class_accuarcy)
            return test_class_output.cpu().numpy(), test_feature.cpu().numpy()

    model = Similarity_AU_model()
    model.train()





    # Evaluate on Test Set



    #model = Similarity_AU_model()
    #model.load_state_dict(torch.load('./model_relabeled/tripletloss/happiness_model_forcing_similarity_by_Adjusted_AU_triplet_100.pth'))


    trans = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    face_names = []
    scores = []
    preds = []

    true_labels = []
    true_gender_labels = []

    for index, row in test_frame.iterrows():

        if index % 200 == 0:
            print(index)

        image_name = row['ImageName']

        image = dlib.load_rgb_image(image_name)
        image = trans(image)
        image = image.view(1, 3, 224, 224)
        image = image.to(device)

        outputs = model.public_test(image)
        outputs = outputs.cpu().detach().numpy()
        outputs = np.squeeze(outputs)

        score = outputs * 1
        #score = np.exp(outputs) / np.sum(np.exp(outputs))
        #pred = (score>=0.5)*1
        pred = np.argmax(score)

        face_names.append(image_name)
        scores.append(score)
        preds.append(pred)

        true_labels.append(row['Expression'])
        true_gender_labels.append(row['gender_preds'])

    test_result = pd.DataFrame(list(zip(face_names, scores, preds, true_labels, true_gender_labels)), 
                 columns = ['ImageName', 'ExpressionScore', 'Prediction', 'Expression',  'Gender'])
    test_result.head(10)

    # Accuracy
    test_result['CorrectOrNot'] = (test_result.Prediction == test_result.Expression)


    dict_row = {}
    dict_row['model_random_state'] = k
    dict_row['test_accuracy'] = test_result.CorrectOrNot.mean()
    dict_row['test_male_true_proportion'] = (test_result[test_result.Gender == "Male"].Expression==1).mean()
    dict_row['test_female_true_proportion'] = (test_result[test_result.Gender == "Female"].Expression==1).mean()
    dict_row['test_male_predicted_proportion'] = (test_result[test_result.Gender == "Male"].Prediction==1).mean()
    dict_row['test_female_predicted_proportion'] = (test_result[test_result.Gender == "Female"].Prediction==1).mean()
    dict_row['test_male_average_score'] = test_result[test_result.Gender == "Male"].ExpressionScore.apply(lambda x: x[1]).mean()
    dict_row['test_female_average_score'] = test_result[test_result.Gender == "Female"].ExpressionScore.apply(lambda x: x[1]).mean()

    # Apply on Chicago Face

    chicago_df = pd.read_csv('ChicagoFace_selected_evaluation_set2.csv')
    chicago_df.head()

    def expression_string_to_num(s):
        if s == "Happy":
            return 3
        elif s == "Angry":
            return 0
        elif s == "Fear":
            return 2
        else:
            return 6

    chicago_df["Expression_num"] = chicago_df.Expression.apply(expression_string_to_num)


    chicago_df["happy"] = chicago_df.Expression_num.apply(lambda x: 1 if x == 3 else 0)


    
    #model = Similarity_AU_model()
    #model.load_state_dict(torch.load('./model_relabeled/tripletloss/happiness_model_forcing_similarity_by_Adjusted_AU_triplet_100.pth'))



    trans = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    face_names = []
    scores = []
    preds = []

    true_labels = []

    for index, row in chicago_df.iterrows():

        #if index >= 100: break

        if index % 100 == 0:
            print(index)

        image_name = row['ImageName']


        image = dlib.load_rgb_image(image_name)
        image = trans(image)
        image = image.view(1, 3, 224, 224)
        image = image.to(device)

        outputs = model.public_test(image)
        outputs = outputs.cpu().detach().numpy()
        outputs = np.squeeze(outputs)

        score = outputs * 1
        #score = np.exp(outputs) / np.sum(np.exp(outputs))
        #pred = (score>=0.5)*1
        pred = np.argmax(score)

        face_names.append(image_name)
        scores.append(score)
        preds.append(pred)

        true_labels.append(row['happy'])

    chicago_result = pd.DataFrame([face_names, true_labels, preds, scores]).T

    chicago_result.columns = ['ImageName', 'Expression_num', 'expression_preds', 'expression_scores']



    chicago_result["happiness_score"] = chicago_result.expression_scores.apply(lambda x: x[1])


    num_happy = (chicago_df.happy == 1).sum()
    threshold = chicago_result.sort_values(by = ['happiness_score'], ascending = False).reset_index(drop = True).iloc[num_happy]['happiness_score']

    chicago_result['expression_preds_relabeled'] = chicago_result.happiness_score.apply(lambda x : 1 if x > threshold else 0)


    chicago_result['CorrectOrNot'] = (chicago_result.expression_preds_relabeled == chicago_result.Expression_num)
    dict_row['selected_2_chicago_accuracy'] = chicago_result.CorrectOrNot.mean() 

    chicago_df_merged = pd.merge(chicago_df, chicago_result, on = ['ImageName'], how = 'left')
    chicago_df_merged.columns = ['ImageName', 'Gender', 'Race', 'Expression', 'AU06_r_c', 'AU12_r_c',
           'AU', 'Expression_num_6','happy', 'Expression_num', 'expression_preds', 'expression_scores',
           'happiness_score','expression_preds_relabeled', 'CorrectOrNot']
    chicago_df_merged.head()

    pd.crosstab(chicago_result.Expression_num, chicago_result.expression_preds_relabeled)

    chicago_male_frame = chicago_df_merged.loc[chicago_df_merged['Gender'] == 'M']
    chicago_female_frame = chicago_df_merged.loc[chicago_df_merged['Gender'] == 'F']

    # Accuracy between males and females
    dict_row['selected_2_chicago_accuracy_male'] = (chicago_male_frame.expression_preds_relabeled == chicago_male_frame.Expression_num).mean()
    dict_row['selected_2_chicago_accuracy_female'] = (chicago_female_frame.expression_preds_relabeled == chicago_female_frame.Expression_num).mean()

    # True proportion
    dict_row['selected_2_chicago_true_proportion'] = chicago_male_frame.happy.mean()

    # Prediction proportion
    dict_row['selected_2_chicago_male_predicted_proportion'] = chicago_male_frame.expression_preds_relabeled.mean()
    dict_row['selected_2_chicago_female_predicted_proportion'] = chicago_female_frame.expression_preds_relabeled.mean()
    dict_row['selected_2_chicago_bias'] = chicago_female_frame.expression_preds_relabeled.mean() - chicago_male_frame.expression_preds_relabeled.mean()

    # Prediction proportion
    dict_row['selected_2_chicago_male_predicted_proportion_raw'] = chicago_male_frame.expression_preds.mean()
    dict_row['selected_2_chicago_female_predicted_proportion_raw'] = chicago_female_frame.expression_preds.mean()
    dict_row['selected_2_chicago_bias_raw'] = chicago_female_frame.expression_preds.mean() - chicago_male_frame.expression_preds.mean()

    # Average Happiness Score
    dict_row['selected_2_chicago_male_score'] = chicago_male_frame.happiness_score.mean()
    dict_row['selected_2_chicago_female_score'] = chicago_female_frame.happiness_score.mean()

    # Average Happiness Score among neutral faces
    dict_row['selected_2_chicago_male_score_neutral_faces'] = chicago_male_frame[chicago_male_frame.Expression_num_6 == 6].happiness_score.mean()
    dict_row['selected_2_chicago_female_score_neutral_faces'] = chicago_female_frame[chicago_female_frame.Expression_num_6 == 6].happiness_score.mean()

    result_rows_list.append(dict_row)
    results_df = pd.DataFrame(result_rows_list) 
    print(results_df)
    results_df.to_csv('model_evaluation_result.csv', index = False)
    
    del model
    torch.cuda.empty_cache()

16000 train faces, 2000 validation faces, 2000 test faces

learning rate: 0.0001
epoch number: 6

Training epoch 0: [500|500], class loss:100.53656768798828, class accuracy: 77.8625
Testing epoch, class loss:100.26173098125155,  class accuracy: 83.9

Training epoch 1: [500|500], class loss:90.5214614868164, class accuracy: 81.15625
Testing epoch, class loss:102.87454175192212,  class accuracy: 83.4

Training epoch 2: [500|500], class loss:95.02030181884766, class accuracy: 81.49375
Testing epoch, class loss:100.75304315345628,  class accuracy: 84.35

Training epoch 3: [500|500], class loss:90.80130004882812, class accuracy: 81.775
Testing epoch, class loss:100.40850733196925,  class accuracy: 83.75

Training epoch 4: [500|500], class loss:94.13863372802734, class accuracy: 82.075
Testing epoch, class loss:93.33293054311994,  class accuracy: 83.95

Training epoch 5: [500|500], class loss:89.98052978515625, class accuracy: 82.2
Testing epoch, class loss:96.73107916211325,  class accuracy


learning rate: 0.0001
epoch number: 6

Training epoch 0: [500|500], class loss:110.74781799316406, class accuracy: 77.8375
Testing epoch, class loss:103.62725188240172,  class accuracy: 81.6

Training epoch 1: [500|500], class loss:88.45552062988281, class accuracy: 80.85
Testing epoch, class loss:96.32533409481957,  class accuracy: 82.45

Training epoch 2: [500|500], class loss:96.01760864257812, class accuracy: 82.21875
Testing epoch, class loss:100.18267349969773,  class accuracy: 82.35

Training epoch 3: [500|500], class loss:95.25269317626953, class accuracy: 81.575
Testing epoch, class loss:101.77506704179068,  class accuracy: 83.5

Training epoch 4: [500|500], class loss:92.05667877197266, class accuracy: 81.78125
Testing epoch, class loss:96.13916621132502,  class accuracy: 83.65

Training epoch 5: [500|500], class loss:90.43923950195312, class accuracy: 82.1625
Testing epoch, class loss:99.07972039116754,  class accuracy: 82.35

learning rate: 1e-05
epoch number: 4

Training 


learning rate: 0.0001
epoch number: 6

Training epoch 0: [500|500], class loss:91.54320526123047, class accuracy: 77.79375
Testing epoch, class loss:101.51878369043744,  class accuracy: 82.5

Training epoch 1: [500|500], class loss:101.10200500488281, class accuracy: 81.3625
Testing epoch, class loss:97.48912678067646,  class accuracy: 81.75

Training epoch 2: [500|500], class loss:88.6354751586914, class accuracy: 81.43125
Testing epoch, class loss:98.60257321312314,  class accuracy: 82.45

Training epoch 3: [500|500], class loss:99.6183090209961, class accuracy: 81.3125
Testing epoch, class loss:100.52017393566314,  class accuracy: 79.65

Training epoch 4: [500|500], class loss:104.07324981689453, class accuracy: 81.96875
Testing epoch, class loss:97.84797898549883,  class accuracy: 82.3

Training epoch 5: [500|500], class loss:102.65785217285156, class accuracy: 82.025
Testing epoch, class loss:98.4452859787714,  class accuracy: 83.25

learning rate: 1e-05
epoch number: 4

Training